<a href="https://colab.research.google.com/github/dipta007/Movie-Recommendation/blob/multi-label-classification/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from os.path import join
import os

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/Colab_data/MR1'       # path to your project on Drive
drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   
%cd "{PROJECT_PATH}"
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/Colab_data/MR1’: File exists
/content/drive/My Drive/Colab_data/MR1
data/  multilayer_perceptron_graph.png


In [2]:
%matplotlib inline
import numpy as np
import pandas as pd

DATA_PATH = './data/ml-latest-small'

ratings = pd.read_csv(f"{DATA_PATH}/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies = pd.read_csv(f"{DATA_PATH}/movies.csv")
movies_ind_to_name = dict(zip(movies.movieId, movies.title))
movies_name_to_ind = dict(zip(movies.title, movies.movieId))
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

user_enc = LabelEncoder()
ratings['user'] = user_enc.fit_transform(ratings['userId'].values)
n_users = ratings['user'].nunique()

movie_enc = LabelEncoder()
movie_enc.fit(movies['movieId'])
ratings['movie'] = movie_enc.transform(ratings['movieId'].values)
n_movies = movies['movieId'].nunique()

ratings['rating'] = ratings['rating'].values.astype(np.float32)
min_rating = min(ratings['rating'])
max_rating = max(ratings['rating'])

# n_users, n_movies, min_rating, max_rating
# del ratings['timestamp']
ratings.head()
ratings.shape

(100836, 6)

In [5]:
n_users, n_movies

(610, 9742)

In [6]:
# Delete all the rows with less ratings
indexes = ratings[ ratings.rating < 3.5 ].index
nratings = ratings.drop(indexes)
nratings.shape

(61716, 6)

In [0]:
user_movie = {}
for index, row in nratings.iterrows():
  now = int(row.user)
  if now not in user_movie:
    user_movie[now] = []
  user_movie[now].append((row.movie, row.rating))

MIN_NUMBER_OF_MOVIES = 4

In [10]:
MIN_NUMBER_OF_MOVIES, n_users, n_movies, min_rating, max_rating

(4, 610, 9742, 0.5, 5.0)

In [0]:
def convert_to_one_hot(movies):
  now = [0 for _ in range(n_movies)]
  for movie, rating in movies:
    now[int(movie)] = 1.0
  return now

prev_movies = []
next_movies = []
for user in range(n_users):
  if user not in user_movie:
    continue
  user_movie[user].sort(key=lambda x: x[1])
  for i in range(len(user_movie[user]) - MIN_NUMBER_OF_MOVIES - 1):
    now_movies = user_movie[user]
    
    # Previous movies he has saw already
    prev = [mov for (mov, rat) in now_movies[i:i+MIN_NUMBER_OF_MOVIES]]
    prev_movies.append(prev);

    # Next movies he should see
    next_movies.append(convert_to_one_hot(now_movies[i+MIN_NUMBER_OF_MOVIES:]));


In [14]:
len(prev_movies), len(next_movies)

(58682, 58682)

In [15]:
X = np.array(prev_movies).reshape(len(prev_movies), 4, 1)
Y = np.array(next_movies)
X.shape, Y.shape

((58682, 4, 1), (58682, 9742))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=47)

In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((46945, 4, 1), (11737, 4, 1), (46945, 9742), (11737, 9742))

In [18]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l2
from keras.layers import Concatenate, Dense, Dropout, Activation, Lambda, LSTM

def Model1(n_users, n_movies):
  input1 = Input(shape=(MIN_NUMBER_OF_MOVIES, 1,))
  emb = Embedding(50)(input1)
  x = LSTM(128)(emb)
  x = Dense(1000)(x)
  x = Dropout(2)(x)
  x = Dense(n_movies)(x)
  x = Activation('sigmoid')(x)

  model = Model(inputs=input1, outputs=x)
  # rms = RMSprop(lr=0.001)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  
  # model.compile(loss='mean_squared_error', optimizer=rms, metrics=['accuracy'])
  return model

Using TensorFlow backend.


In [19]:
model = Model1(n_users, n_movies)
model.summary()

# from keras.utils import plot_model
# plot_model(model, to_file='multilayer_perceptron_graph.png')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 1)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              129000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 9742)              9751742   
_________________________________________________________________
activation_1 (Activation)    (None, 9742)              0         
Total params: 9,947,302
Trainable params: 9,947,302
Non-trainable params: 0
_________________________________________________

In [20]:
history = model.fit(x=X, y=Y, batch_size=64, epochs=5, verbose=1, 
                    validation_data=(X_test, y_test))

Train on 58682 samples, validate on 11737 samples
Epoch 1/5
58682/58682 [==============================] - 51s 874us/step - loss: 0.0703 - accuracy: 0.9822 - val_loss: 0.0663 - val_accuracy: 0.9827
Epoch 2/5
58682/58682 [==============================] - 51s 868us/step - loss: 0.0656 - accuracy: 0.9829 - val_loss: 0.0661 - val_accuracy: 0.9827
Epoch 3/5
58682/58682 [==============================] - 51s 872us/step - loss: 0.0654 - accuracy: 0.9829 - val_loss: 0.0657 - val_accuracy: 0.9827
Epoch 4/5
58682/58682 [==============================] - 51s 867us/step - loss: 0.0656 - accuracy: 0.9829 - val_loss: 0.0662 - val_accuracy: 0.9827
Epoch 5/5
58682/58682 [==============================] - 51s 864us/step - loss: 0.0653 - accuracy: 0.9829 - val_loss: 0.0657 - val_accuracy: 0.9827


In [0]:
def get_result(movie_no, n = 10):
  nolan = movie_enc.transform([109487, 33794, 33794, 74458])
  pred = model.predict(nolan.reshape(1, 4, 1))[0]
  res = []
  for (ind, pred) in enumerate(pred):
      res.append((ind, pred))

  res.sort(key=lambda x: x[1], reverse=True)
  res = [(movie_enc.inverse_transform([id])[0], pred) for (id, pred) in res]
  for (movieId, pred) in res[0:n]:
    print(f"{movies_ind_to_name[movieId]} -> {pred}")
  return None

In [26]:
# godfather, taxidriver, raging bull, goodfellas
get_result([858, 111, 1228, 1213], 10)

Matrix, The (1999) -> 0.5254359245300293
Fight Club (1999) -> 0.4772758185863495
Shawshank Redemption, The (1994) -> 0.42900142073631287
Star Wars: Episode V - The Empire Strikes Back (1980) -> 0.419495552778244
Pulp Fiction (1994) -> 0.4086754620075226
Forrest Gump (1994) -> 0.36886096000671387
Godfather, The (1972) -> 0.3557226061820984
Inception (2010) -> 0.35343167185783386
Good Will Hunting (1997) -> 0.35331833362579346
Silence of the Lambs, The (1991) -> 0.3476870059967041


In [27]:
# eat pray love, Eternal Sunshine of the Spotless Mind, the intern, The notebook
get_result([80551, 7361, 140110, 8533], 10)

Matrix, The (1999) -> 0.5254359245300293
Fight Club (1999) -> 0.4772758185863495
Shawshank Redemption, The (1994) -> 0.42900142073631287
Star Wars: Episode V - The Empire Strikes Back (1980) -> 0.419495552778244
Pulp Fiction (1994) -> 0.4086754620075226
Forrest Gump (1994) -> 0.36886096000671387
Godfather, The (1972) -> 0.3557226061820984
Inception (2010) -> 0.35343167185783386
Good Will Hunting (1997) -> 0.35331833362579346
Silence of the Lambs, The (1991) -> 0.3476870059967041


In [24]:
# into the wild, seven years in tibet, cast away, the pursuit of happyness
get_result([55247, 1619, 4022, 8131], 10)

Matrix, The (1999)
Fight Club (1999)
Shawshank Redemption, The (1994)
Star Wars: Episode V - The Empire Strikes Back (1980)
Pulp Fiction (1994)
Forrest Gump (1994)
Godfather, The (1972)
Inception (2010)
Good Will Hunting (1997)
Silence of the Lambs, The (1991)


In [0]:
for thresh in [0.4, 0.3]:
  res = get_result(pred[0], thresh)
  print(len(res))
  print(res)

In [0]:
movie_enc.inverse_transform([7355, 8045, 8457, 8528, 8618, 8663, 8668, 8882])